In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text
df_stock = pd.read_csv("tata_steel/TATASTEEL.NS (1).csv",
                 parse_dates = ["Date"],
                 index_col = ["Date"])
df_posts = pd.read_excel("tata_steel/tata_steel_final_posts.xlsx")
# removing the unnecessary columns
df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
# removing spam posts
df_posts = df_posts[df_posts.Spam==0.0]
df_posts.drop(["Spam"],axis=1,inplace=True)
# sliding a window of 7 days and adding all the TIs
from stock_helper import prepare_data
x,y = prepare_data(df_stock)
final_x = x[np.datetime64("2021-11-13"):]
final_y = y[np.datetime64("2021-11-13"):]
# reversing the posts data
df_posts = df_posts[::-1]
final_posts = df_posts[7:]
# loading the sentiment analysis model
final_x_zeros = final_x.copy()
final_y_zeros = final_y.copy()
final_y_zeros = final_y_zeros.to_frame()

In [2]:
final_x_zeros

,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACDh_12_26_9,RSI_14,AROONOSC_14,Close + 1,BBL_20_2.0 + 1,...,AROONOSC_14 + 6,Close + 7,BBL_20_2.0 + 7,BBM_20_2.0 + 7,BBU_20_2.0 + 7,BBB_20_2.0 + 7,BBP_20_2.0 + 7,MACDh_12_26_9 + 7,RSI_14 + 7,AROONOSC_14 + 7
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-15,1253.773926,1322.569946,1391.366089,10.403396,-0.060497,-6.295380,37.464535,-64.285713,1287.250000,1266.390015,...,-28.571428,1326.050049,1262.338379,1327.867554,1393.396729,9.869839,0.486132,1.445773,49.121861,-28.571428
2021-11-16,1245.681641,1313.457520,1381.233398,10.320224,-0.124540,-9.121670,35.462971,-71.428574,1245.449951,1253.773926,...,-28.571428,1325.900024,1268.413818,1330.167480,1391.921143,9.285093,0.465447,1.373855,49.100006,-28.571428
2021-11-17,1231.605347,1306.114990,1380.624634,11.409358,-0.077542,-10.887480,34.422169,-78.571426,1228.800049,1245.681641,...,-28.571428,1343.250000,1273.854248,1332.869995,1391.885742,8.855443,0.587942,2.416823,51.772297,-28.571428
2021-11-18,1209.474854,1298.280029,1387.085205,13.680439,-0.128792,-13.447521,30.711382,-85.714287,1220.050049,1231.605347,...,-21.428572,1335.949951,1277.413696,1334.609985,1391.806274,8.571238,0.511714,2.525293,50.569298,-28.571428
2021-11-22,1185.163574,1290.797485,1396.431519,16.367241,-0.095204,-15.581080,28.574228,-92.857140,1186.599976,1209.474854,...,35.714287,1298.550049,1275.245972,1333.877563,1392.509033,8.791142,0.198733,0.115691,44.822891,-21.428572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-07,1069.180298,1160.972534,1252.764648,15.812980,0.619986,4.002861,55.514107,-50.000000,1176.300049,1068.605591,...,-50.000000,1088.349976,1074.149414,1159.670044,1245.190552,14.749115,0.083024,-9.462365,38.504116,-50.000000
2022-02-08,1068.239136,1163.500000,1258.760864,16.374884,0.794454,7.265931,61.140900,-42.857143,1183.000000,1069.180298,...,-50.000000,1084.650024,1070.828369,1158.852539,1246.876709,15.191608,0.078511,-11.377896,38.033764,-50.000000
2022-02-09,1070.835449,1168.104980,1265.374512,16.654247,0.778839,9.120802,61.534584,64.285713,1219.599976,1068.239136,...,-50.000000,1085.550049,1066.087280,1157.557495,1249.027710,15.803999,0.106388,-11.926422,38.231426,-50.000000


In [9]:
tf.random.set_seed(42)
inputs = tf.keras.Input(shape=(71,1), name="inputs")
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(inputs)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(x)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.models.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 71, 1)]           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 71, 64)            256       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 35, 64)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 35, 64)            12352     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 17, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0   

In [10]:
model.compile(loss = "mae", optimizer=tf.keras.optimizers.Adam(), 
              metrics=["mae","mse"])
history = model.fit(final_x_zeros[:42],final_y_zeros[:42],
                    epochs = 1000,
                    validation_data=(final_x_zeros[42:],final_y_zeros[42:]), 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                            patience=200, 
                                                            restore_best_weights=True),
                          tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                               patience=100, 
                                                               verbose=1)])

Epoch 1/1000
2/2 [==============================] - 0s 144ms/step - loss: 1265.3225 - mae: 1265.3225 - mse: 1605749.3750 - val_loss: 1068.6147 - val_mae: 1068.6147 - val_mse: 1144717.7500 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 36ms/step - loss: 1004.2138 - mae: 1004.2138 - mse: 1013231.5000 - val_loss: 829.1525 - val_mae: 829.1525 - val_mse: 690333.8750 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 32ms/step - loss: 758.8659 - mae: 758.8659 - mse: 579066.1875 - val_loss: 601.3857 - val_mae: 601.3857 - val_mse: 364566.5625 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 28ms/step - loss: 524.6414 - mae: 524.6414 - mse: 280006.2812 - val_loss: 379.9466 - val_mae: 379.9466 - val_mse: 147282.8594 - lr: 0.0010
Epoch 5/1000
2/2 [==============================] - 0s 32ms/step - loss: 295.2096 - mae: 295.2096 - mse: 92793.1094 - val_loss: 160.0511 - val_mae: 160.0511 - val_mse: 28599.9336 - lr: 0.0010
Epoch 6/1000
2/2 [====

2/2 [==============================] - 0s 28ms/step - loss: 47.2308 - mae: 47.2308 - mse: 3322.0249 - val_loss: 65.4344 - val_mae: 65.4344 - val_mse: 4834.3945 - lr: 0.0010
Epoch 45/1000
2/2 [==============================] - 0s 24ms/step - loss: 46.7833 - mae: 46.7833 - mse: 3331.4104 - val_loss: 61.8463 - val_mae: 61.8463 - val_mse: 4280.8896 - lr: 0.0010
Epoch 46/1000
2/2 [==============================] - 0s 24ms/step - loss: 46.3997 - mae: 46.3997 - mse: 3569.1719 - val_loss: 61.0287 - val_mae: 61.0287 - val_mse: 4163.9971 - lr: 0.0010
Epoch 47/1000
2/2 [==============================] - 0s 28ms/step - loss: 46.0461 - mae: 46.0461 - mse: 3477.4207 - val_loss: 63.8980 - val_mae: 63.8980 - val_mse: 4570.0742 - lr: 0.0010
Epoch 48/1000
2/2 [==============================] - 0s 28ms/step - loss: 45.6883 - mae: 45.6883 - mse: 3161.9111 - val_loss: 64.2677 - val_mae: 64.2677 - val_mse: 4622.5884 - lr: 0.0010
Epoch 49/1000
2/2 [==============================] - 0s 28ms/step - loss: 45.20

Epoch 88/1000
2/2 [==============================] - 0s 28ms/step - loss: 37.1917 - mae: 37.1917 - mse: 1970.7487 - val_loss: 66.6349 - val_mae: 66.6349 - val_mse: 5001.4077 - lr: 0.0010
Epoch 89/1000
2/2 [==============================] - 0s 28ms/step - loss: 34.5333 - mae: 34.5333 - mse: 1698.7599 - val_loss: 63.4206 - val_mae: 63.4206 - val_mse: 4485.2695 - lr: 0.0010
Epoch 90/1000
2/2 [==============================] - 0s 28ms/step - loss: 33.5929 - mae: 33.5929 - mse: 1678.1494 - val_loss: 58.5698 - val_mae: 58.5698 - val_mse: 3782.9011 - lr: 0.0010
Epoch 91/1000
2/2 [==============================] - 0s 32ms/step - loss: 32.9965 - mae: 32.9965 - mse: 1646.0537 - val_loss: 62.7163 - val_mae: 62.7163 - val_mse: 4382.1362 - lr: 0.0010
Epoch 92/1000
2/2 [==============================] - 0s 28ms/step - loss: 32.4805 - mae: 32.4805 - mse: 1577.6829 - val_loss: 63.5020 - val_mae: 63.5020 - val_mse: 4503.8340 - lr: 0.0010
Epoch 93/1000
2/2 [==============================] - 0s 28ms/step

Epoch 132/1000
2/2 [==============================] - 0s 24ms/step - loss: 37.1214 - mae: 37.1214 - mse: 2070.5898 - val_loss: 58.5034 - val_mae: 58.5034 - val_mse: 3809.4573 - lr: 0.0010
Epoch 133/1000
2/2 [==============================] - 0s 28ms/step - loss: 28.2596 - mae: 28.2596 - mse: 1193.2013 - val_loss: 50.7823 - val_mae: 50.7823 - val_mse: 2941.6863 - lr: 0.0010
Epoch 134/1000
2/2 [==============================] - 0s 28ms/step - loss: 31.1484 - mae: 31.1484 - mse: 1505.5198 - val_loss: 58.2007 - val_mae: 58.2007 - val_mse: 3767.0051 - lr: 0.0010
Epoch 135/1000
2/2 [==============================] - 0s 28ms/step - loss: 28.6568 - mae: 28.6568 - mse: 1232.9747 - val_loss: 65.4208 - val_mae: 65.4208 - val_mse: 4962.9116 - lr: 0.0010
Epoch 136/1000
2/2 [==============================] - 0s 24ms/step - loss: 31.1338 - mae: 31.1338 - mse: 1399.8103 - val_loss: 57.7750 - val_mae: 57.7750 - val_mse: 3707.4902 - lr: 0.0010
Epoch 137/1000
2/2 [==============================] - 0s 28m

Epoch 176/1000
2/2 [==============================] - 0s 28ms/step - loss: 25.3389 - mae: 25.3389 - mse: 994.0536 - val_loss: 53.0731 - val_mae: 53.0731 - val_mse: 3123.8289 - lr: 0.0010
Epoch 177/1000
2/2 [==============================] - 0s 28ms/step - loss: 26.8744 - mae: 26.8744 - mse: 1117.3119 - val_loss: 57.9084 - val_mae: 57.9084 - val_mse: 3780.2571 - lr: 0.0010
Epoch 178/1000
2/2 [==============================] - 0s 28ms/step - loss: 25.9920 - mae: 25.9920 - mse: 1010.7210 - val_loss: 65.5006 - val_mae: 65.5006 - val_mse: 5121.8442 - lr: 0.0010
Epoch 179/1000
2/2 [==============================] - 0s 24ms/step - loss: 29.2507 - mae: 29.2507 - mse: 1264.1127 - val_loss: 55.0559 - val_mae: 55.0559 - val_mse: 3359.5144 - lr: 0.0010
Epoch 180/1000
2/2 [==============================] - 0s 24ms/step - loss: 25.3955 - mae: 25.3955 - mse: 1057.3170 - val_loss: 52.6944 - val_mae: 52.6944 - val_mse: 3076.4771 - lr: 0.0010
Epoch 181/1000
2/2 [==============================] - 0s 28ms

Epoch 220/1000
2/2 [==============================] - 0s 28ms/step - loss: 29.3556 - mae: 29.3556 - mse: 1283.5574 - val_loss: 62.9480 - val_mae: 62.9480 - val_mse: 4665.9038 - lr: 0.0010
Epoch 221/1000
2/2 [==============================] - 0s 28ms/step - loss: 25.0410 - mae: 25.0410 - mse: 1017.3202 - val_loss: 59.7690 - val_mae: 59.7690 - val_mse: 4127.2207 - lr: 0.0010
Epoch 222/1000
2/2 [==============================] - 0s 24ms/step - loss: 23.2665 - mae: 23.2665 - mse: 862.2598 - val_loss: 52.5155 - val_mae: 52.5155 - val_mse: 3051.7029 - lr: 0.0010
Epoch 223/1000
2/2 [==============================] - 0s 28ms/step - loss: 24.8787 - mae: 24.8787 - mse: 998.2394 - val_loss: 58.4907 - val_mae: 58.4907 - val_mse: 3933.1157 - lr: 0.0010
Epoch 224/1000
2/2 [==============================] - 0s 28ms/step - loss: 25.6749 - mae: 25.6749 - mse: 1023.6440 - val_loss: 62.4725 - val_mae: 62.4725 - val_mse: 4614.1860 - lr: 0.0010
Epoch 225/1000
2/2 [==============================] - 0s 28ms/

Epoch 264/1000
2/2 [==============================] - 0s 28ms/step - loss: 24.4824 - mae: 24.4824 - mse: 967.7557 - val_loss: 61.1034 - val_mae: 61.1034 - val_mse: 4463.8774 - lr: 0.0010
Epoch 265/1000
2/2 [==============================] - 0s 28ms/step - loss: 22.9106 - mae: 22.9106 - mse: 836.0502 - val_loss: 46.4111 - val_mae: 46.4111 - val_mse: 2559.7466 - lr: 0.0010
Epoch 266/1000
2/2 [==============================] - 0s 28ms/step - loss: 30.4845 - mae: 30.4845 - mse: 1415.6084 - val_loss: 53.3245 - val_mae: 53.3245 - val_mse: 3169.5122 - lr: 0.0010
Epoch 267/1000
2/2 [==============================] - 0s 24ms/step - loss: 21.3173 - mae: 21.3173 - mse: 818.9694 - val_loss: 67.4246 - val_mae: 67.4246 - val_mse: 5682.1333 - lr: 0.0010
Epoch 268/1000
2/2 [==============================] - 0s 28ms/step - loss: 30.3698 - mae: 30.3698 - mse: 1418.2123 - val_loss: 55.0782 - val_mae: 55.0782 - val_mse: 3400.7644 - lr: 0.0010
Epoch 269/1000
2/2 [==============================] - 0s 28ms/s

Epoch 308/1000
2/2 [==============================] - 0s 28ms/step - loss: 20.7494 - mae: 20.7494 - mse: 751.0116 - val_loss: 59.9278 - val_mae: 59.9278 - val_mse: 4330.5469 - lr: 0.0010
Epoch 309/1000
2/2 [==============================] - 0s 28ms/step - loss: 22.4327 - mae: 22.4327 - mse: 869.4180 - val_loss: 54.9059 - val_mae: 54.9059 - val_mse: 3426.8125 - lr: 0.0010
Epoch 310/1000
2/2 [==============================] - 0s 28ms/step - loss: 23.2809 - mae: 23.2809 - mse: 924.4896 - val_loss: 52.7147 - val_mae: 52.7147 - val_mse: 3114.7292 - lr: 0.0010
Epoch 311/1000
1/2 [==============>...............] - ETA: 0s - loss: 23.8498 - mae: 23.8498 - mse: 899.9803
Epoch 00311: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
2/2 [==============================] - 0s 28ms/step - loss: 24.9247 - mae: 24.9247 - mse: 933.5847 - val_loss: 58.7455 - val_mae: 58.7455 - val_mse: 4103.1733 - lr: 0.0010
Epoch 312/1000
2/2 [==============================] - 0s 28ms/step - loss: 21

2/2 [==============================] - 0s 28ms/step - loss: 19.7396 - mae: 19.7396 - mse: 704.3139 - val_loss: 57.0002 - val_mae: 57.0002 - val_mse: 3798.4688 - lr: 1.0000e-04
Epoch 351/1000
2/2 [==============================] - 0s 24ms/step - loss: 20.1134 - mae: 20.1134 - mse: 727.6619 - val_loss: 56.9529 - val_mae: 56.9529 - val_mse: 3791.2871 - lr: 1.0000e-04
Epoch 352/1000
2/2 [==============================] - 0s 28ms/step - loss: 20.0891 - mae: 20.0891 - mse: 730.9930 - val_loss: 56.0236 - val_mae: 56.0236 - val_mse: 3627.7939 - lr: 1.0000e-04
Epoch 353/1000
2/2 [==============================] - 0s 28ms/step - loss: 19.7973 - mae: 19.7973 - mse: 707.4556 - val_loss: 55.6349 - val_mae: 55.6349 - val_mse: 3562.2031 - lr: 1.0000e-04
Epoch 354/1000
2/2 [==============================] - 0s 28ms/step - loss: 19.9105 - mae: 19.9105 - mse: 709.4711 - val_loss: 55.3868 - val_mae: 55.3868 - val_mse: 3520.8066 - lr: 1.0000e-04
Epoch 355/1000
2/2 [==============================] - 0s 24m

Epoch 393/1000
2/2 [==============================] - 0s 28ms/step - loss: 19.6424 - mae: 19.6424 - mse: 708.2538 - val_loss: 57.2630 - val_mae: 57.2630 - val_mse: 3850.2620 - lr: 1.0000e-04
Epoch 394/1000
2/2 [==============================] - 0s 28ms/step - loss: 19.9066 - mae: 19.9066 - mse: 720.5673 - val_loss: 57.7935 - val_mae: 57.7935 - val_mse: 3951.0334 - lr: 1.0000e-04
Epoch 395/1000
2/2 [==============================] - 0s 28ms/step - loss: 20.1366 - mae: 20.1366 - mse: 731.5929 - val_loss: 57.6799 - val_mae: 57.6799 - val_mse: 3929.2449 - lr: 1.0000e-04
Epoch 396/1000
2/2 [==============================] - 0s 28ms/step - loss: 20.0350 - mae: 20.0350 - mse: 728.6536 - val_loss: 57.1669 - val_mae: 57.1669 - val_mse: 3832.1504 - lr: 1.0000e-04
Epoch 397/1000
2/2 [==============================] - 0s 28ms/step - loss: 19.7664 - mae: 19.7664 - mse: 714.4989 - val_loss: 56.0220 - val_mae: 56.0220 - val_mse: 3627.3254 - lr: 1.0000e-04
Epoch 398/1000
2/2 [=========================

In [11]:
model.evaluate(final_x_zeros[42:],final_y_zeros[42:])

1/1 [==============================] - 0s 20ms/step - loss: 43.6256 - mae: 43.6256 - mse: 2491.0745


[43.62564468383789, 43.62564468383789, 2491.074462890625]

In [12]:
model.save("conv1d_tata steel")

INFO:tensorflow:Assets written to: conv1d_tata steel\assets
